In [1]:
import os
import nltk
import re
import operator
import gensim 
import numpy as np
from stemming.porter2 import stem
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

In [2]:
dictionary = {}
dictionary_index = {}
exp_path_pos = '../review_polarity/txt_sentoken/pos/'
exp_path_neg = '../review_polarity/txt_sentoken/neg/'
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))
stopW_without_punctuation = []

documents = list()

for i in stopWords:
    stopW_without_punctuation.append(re.sub(r'[^\w\s]','',i))

for filename in os.listdir(exp_path_pos):
    with open(exp_path_pos+filename,'r') as f:
        for line in f:
            #delete Punctuation
            line = re.sub(r'[^\w\s]','',line)
            tempLine = ""
            for word in line.split():
                word = word.lower()
                #Stop Words
                if word not in stopW_without_punctuation:
                    temp_w = stem(word)
                    tempLine += temp_w + " "
                    if temp_w in dictionary:
                        dictionary[temp_w]=dictionary[temp_w] + 1
                    else:
                        dictionary[temp_w]=1
                    
            tempLine = tempLine.strip()
            #El preprocesado simple que trae genism elimina palabras, por lo que cambia el diccionario, lo que hace menos objetiva la comparación    
            #documents.append(gensim.utils.simple_preprocess(tempLine))
            documents.append(tempLine.split())
            
for filename in os.listdir(exp_path_neg):
    with open(exp_path_neg+filename,'r') as f:
        for line in f:
            #delete Punctuation
            line = re.sub(r'[^\w\s]','',line)
            tempLine = ""
            for word in line.split():
                word = word.lower()
                #Stop Words
                if word not in stopW_without_punctuation:
                    temp_w = stem(word)
                    tempLine += temp_w + " "
                    if temp_w in dictionary:
                        dictionary[temp_w]=dictionary[temp_w] + 1
                    else:
                        dictionary[temp_w]=1

            tempLine = tempLine.strip()        
           
            #El preprocesado simple que trae genism elimina palabras, por lo que cambia el diccionario, lo que hace menos objetiva la comparación    
            #documents.append(gensim.utils.simple_preprocess(tempLine))
            documents.append(tempLine.split())

print(len(dictionary))    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/feliperojos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
31749


In [3]:
# Mas frecuentes y menos frecuentes
sorted_dictionary = sorted(dictionary.items(), key=operator.itemgetter(1))
print(sorted_dictionary[0])
print(sorted_dictionary[1])
print(sorted_dictionary[2])
print(sorted_dictionary[3])
print(sorted_dictionary[4])

print(sorted_dictionary[-1])
print(sorted_dictionary[-2])
print(sorted_dictionary[-3])
print(sorted_dictionary[-4])
print(sorted_dictionary[-5])



('santostefano', 1)
('alin', 1)
('brosh', 1)
('romanticmistaken', 1)
('bulimia', 1)
('film', 11108)
('movi', 6855)
('one', 5758)
('like', 3997)
('charact', 3855)


In [4]:
# Preparando para BoW
dictKeys = list(dictionary.keys())
for i in range(len(dictKeys)):
    dictionary_index[dictKeys[i]]=i

    

In [5]:
#codificacion BOW y word2vector
bow_cods = []
w2v_cods = []
labels = []
model = gensim.models.Word2Vec (documents, size=100, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)



(6663263, 7034960)

In [6]:
for filename in os.listdir(exp_path_pos):
    with open(exp_path_pos+filename,'r') as f:
        bow_temp = [0]*len(dictionary)
        w2v_temp = np.empty((0,100))
        count = 0
        for line in f:
            #delete Punctuation
            line = re.sub(r'[^\w\s]','',line)
            tempLine = ""
            for word in line.split():
                word = word.lower()
                #Stop Words
                if word not in stopW_without_punctuation:
                    temp_w = stem(word)
                    bow_temp[dictionary_index[temp_w]]+=1
                    if (temp_w in model.wv.vocab):
                        w2v_resize = np.array(model.wv[temp_w])
                        w2v_resize = np.resize(w2v_resize,(1,100))
                        w2v_temp = np.append(w2v_temp,w2v_resize, axis=0)
        w2v_temp = np.mean(w2v_temp, axis=0)
        bow_cods.append([bow_temp, "pos"])
        w2v_cods.append([w2v_temp, "pos"])
        
for filename in os.listdir(exp_path_neg):
    with open(exp_path_neg+filename,'r') as f:
        bow_temp = [0]*len(dictionary)
        w2v_temp = np.empty((0,100))
        count = 0
        for line in f:
            #delete Punctuation
            line = re.sub(r'[^\w\s]','',line)
            tempLine = ""
            for word in line.split():
                word = word.lower()
                #Stop Words
                if word not in stopW_without_punctuation:
                    temp_w = stem(word)
                    bow_temp[dictionary_index[temp_w]]+=1
                    if (temp_w in model.wv.vocab):
                        w2v_resize = np.array(model.wv[temp_w])
                        w2v_resize = np.resize(w2v_resize,(1,100))
                        w2v_temp = np.append(w2v_temp,w2v_resize, axis=0)
        w2v_temp = np.mean(w2v_temp, axis=0)
        bow_cods.append([bow_temp, "neg"])
        w2v_cods.append([w2v_temp, "neg"])
        

            


In [7]:
kf = KFold(n_splits=5)
folds = []
for i in range(2000):
    folds.append(i)
iterador = 1

for train, test in kf.split(folds):
    print("Iteración numero %i:" % (iterador))
    print("")
    iterador+=1
    lab_fold_train_bow = []
    lab_fold_train_w2v = []
    lab_fold_test_bow = []
    lab_fold_test_w2v = []
    
    con_fold_train_bow = []
    con_fold_train_w2v = []
    con_fold_test_bow = []
    con_fold_test_w2v = []
    for i in train:
        lab_fold_train_bow.append(bow_cods[i][1])
        lab_fold_train_w2v.append(w2v_cods[i][1])
        con_fold_train_bow.append(bow_cods[i][0])
        con_fold_train_w2v.append(w2v_cods[i][0])

    for j in test:
        lab_fold_test_bow.append(bow_cods[j][1])
        lab_fold_test_w2v.append(w2v_cods[j][1])
        con_fold_test_bow.append(bow_cods[j][0])
        con_fold_test_w2v.append(w2v_cods[j][0])
        
    svm_bow = LinearSVC() 
    svm_w2v = LinearSVC()
    
    svm_bow.fit(con_fold_train_bow, lab_fold_train_bow) 
    svm_w2v.fit(con_fold_train_w2v, lab_fold_train_w2v) 
    
    print("Training Set Accuracy BOW:", svm_bow.score(con_fold_train_bow, lab_fold_train_bow))
    print("Testing Set Accuracy BOW:", svm_bow.score(con_fold_test_bow, lab_fold_test_bow))
    print("Training Set Accuracy W2V:", svm_w2v.score(con_fold_train_w2v, lab_fold_train_w2v))
    print("Testing Set Accuracy W2V:", svm_w2v.score(con_fold_test_w2v, lab_fold_test_w2v))
    print("")
    print ('\nConfussion matrix BOW:\n',confusion_matrix(lab_fold_test_bow, svm_bow.predict(con_fold_test_bow)))
    print ('\nConfussion matrix W2V:\n',confusion_matrix(lab_fold_test_w2v, svm_w2v.predict(con_fold_test_w2v)))
    print("")

        
        
        


Iteración numero 1:

Training Set Accuracy BOW: 1.0
Testing Set Accuracy BOW: 0.765
Training Set Accuracy W2V: 0.799375
Testing Set Accuracy W2V: 0.6425


Confussion matrix BOW:
 [[  0   0]
 [ 94 306]]

Confussion matrix W2V:
 [[  0   0]
 [143 257]]

Iteración numero 2:

Training Set Accuracy BOW: 1.0
Testing Set Accuracy BOW: 0.7625
Training Set Accuracy W2V: 0.810625
Testing Set Accuracy W2V: 0.6075


Confussion matrix BOW:
 [[  0   0]
 [ 95 305]]

Confussion matrix W2V:
 [[  0   0]
 [157 243]]

Iteración numero 3:

Training Set Accuracy BOW: 1.0
Testing Set Accuracy BOW: 0.85
Training Set Accuracy W2V: 0.786875
Testing Set Accuracy W2V: 0.7725


Confussion matrix BOW:
 [[178  22]
 [ 38 162]]

Confussion matrix W2V:
 [[158  42]
 [ 49 151]]

Iteración numero 4:

Training Set Accuracy BOW: 1.0
Testing Set Accuracy BOW: 0.75
Training Set Accuracy W2V: 0.800625
Testing Set Accuracy W2V: 0.5725


Confussion matrix BOW:
 [[300 100]
 [  0   0]]

Confussion matrix W2V:
 [[229 171]
 [  0   0]